---
**License**

 gauss_constraint_mcmc

 Mon Jun 26 17:30:00 2023\
 Copyright  2023\
 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

---
---

 gauss_constraint_mcmc\
 Copyright (C) 2023 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
import sys

from numcosmo_py import Ncm

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import numpy as np
import math

import getdist
from getdist import plots
from getdist import MCSamples

from numcosmo_py.experiments.gauss_constraint import run_gauss_constraint_mcmc

from numcosmo_py.plotting.tools import confidence_ellipse
from numcosmo_py.plotting.tools import set_rc_params_article
from numcosmo_py.plotting.getdist import mcat_to_mcsamples
from numcosmo_py.sampling.esmcmc import mcat_print_info

In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

nthreads = 4
nwalkers = 30000
burnin = 49
thin = 1
dim = 50
save_figs = True

Ncm.func_eval_set_max_threads(nthreads)
Ncm.func_eval_log_pool_stats()

In [ ]:
catalog_apes = f"gauss_constraint_{dim}d_mcmc_catalog_WalkerTypes.APES_{nwalkers}.fits"
catalog_tmvn = run_gauss_constraint_mcmc(
    sampler="apes", nwalkers=nwalkers, dim=dim, ssize=1000000, tmvn=True
)
table_tmvn = np.loadtxt(catalog_tmvn)

In [ ]:
mcat_apes = Ncm.MSetCatalog.new_from_file_ro(catalog_apes, nwalkers * burnin)

assert mcat_apes.nchains() == nwalkers

In [ ]:
mset = mcat_apes.peek_mset()
mmvnd = mset.peek(Ncm.ModelMVND.id())
for i in range(dim):
    mmvnd.param_peek_desc(i).props.symbol = rf"x_{{{i+1}}}"

In [ ]:
sample_apes, rows_apes, posterior_apes = mcat_to_mcsamples(
    mcat_apes,
    "APES",
    collapse=True,
)

In [ ]:
names = [pn.name for pn in sample_apes.paramNames.names]
labels = [pn.label for pn in sample_apes.paramNames.names]

posterior_tmvn = 0.5 * table_tmvn[:, 0]
rows_tmvn = np.delete(table_tmvn, 0, 1)

sample_tmvn = MCSamples(
    samples=rows_tmvn, loglikes=posterior_tmvn, names=names, labels=labels, label="TMVN"
)

In [ ]:
set_rc_params_article(ncol=2)

nparams = 8
indices = range(0, nparams)
indices_odd = range(nparams, 2 * nparams, 2)
params = [f"x_{i+1}" for i in indices]

param_limits = {f"x_{i+1}": [0.0 if i % 2 == 0 else -2.0, 2.0] for i in indices}

g = plots.get_subplot_plotter(
    width_inch=plt.rcParams["figure.figsize"][0],
)
g.settings.linewidth = 0.01
g.triangle_plot(
    [sample_tmvn, sample_apes],
    shaded=True,
    params=params,
    param_limits=param_limits,
    diag1d_kwargs={"nbands": 3},
)

if save_figs:
    plt.tight_layout()
    plt.savefig(f"gauss_constraint_{dim}d_corner.pdf")

In [ ]:
kernel = Ncm.StatsDistKernelST.new(dim, 1.0)
interp_vkde = Ncm.StatsDistVKDE.new(kernel, Ncm.StatsDistCV.NONE)
interp_kde = Ncm.StatsDistKDE.new(kernel, Ncm.StatsDistCV.NONE)

# interp_vkde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)
# interp_kde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)
interp_vkde.set_over_smooth(1.1)
interp_kde.set_over_smooth(0.2)

row_kde = rows_tmvn
posterior_kde = posterior_tmvn
label_kde = "tmvn"

In [ ]:
max_n = len(row_kde)
ssize = int(nwalkers / 2)

interp_vkde.reset()
interp_kde.reset()

for theta in row_kde[-ssize:]:
    theta_v = Ncm.Vector.new_array(theta)
    interp_vkde.add_obs(theta_v)
    interp_kde.add_obs(theta_v)

m2lnL_v = Ncm.Vector.new_array(2.0 * posterior_kde[-ssize:])

interp_vkde.prepare_interp(m2lnL_v)
interp_kde.prepare_interp(m2lnL_v)

In [ ]:
set_rc_params_article(ncol=2)
fig, ax = plt.subplots(1, 1)

weights = np.array(interp_vkde.peek_weights().dup_array())
weights = weights / np.sum(weights)
lnweights = np.log(weights)

lnweights = lnweights - np.min(lnweights)
lnweights = lnweights / np.max(lnweights)

pair = [0, 2]
max_w = interp_vkde.peek_weights().get_max()


for i in range(interp_vkde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_vkde.get_Ki(i)
    mu = np.array(y_i.dup_array())[pair]
    cov = np.array([[cov_i.get(i, j) for j in pair] for i in pair])
    cov = cov * 1.0
    confidence_ellipse(mu, cov, ax, facecolor="blue", alpha=w_i / max_w)

ax.set_xlabel("$\mu_0$")
ax.set_ylabel("$\mu_2$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig(f"gauss_constraint_{dim}d_{label_kde}_vkde_kernels.pdf")

In [ ]:
set_rc_params_article(ncol=2)
fig, ax = plt.subplots(1, 1)

max_w = interp_kde.peek_weights().get_max()

for i in range(interp_kde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_kde.get_Ki(i)
    mu = np.array(y_i.dup_array())[pair]
    cov = np.array([[cov_i.get(i, j) for j in pair] for i in pair])
    cov = cov * 1.0

    confidence_ellipse(mu, cov, ax, facecolor="blue", alpha=w_i / max_w)

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig(f"gauss_constraint_{dim}d_{label_kde}_kde_kernels.pdf")

In [ ]:
tmvn_mu = 2.0 * posterior_tmvn
print(np.mean(tmvn_mu))
print(np.std(tmvn_mu))
plt.hist(tmvn_mu, bins=50, density=True, alpha=0.5, label="TMVN")

apes_mu = 2.0 * posterior_apes
print(np.mean(apes_mu))
print(np.std(apes_mu))
plt.hist(apes_mu, bins=50, density=True, alpha=0.5, label="APES")

plt.legend(loc="best")

pass